In [ ]:

import os
import numpy as np
from model import model, dummy_loss
from utils.generator import create_batch
from utils.utils import normalize, evaluate, makedirs,read_annotations
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from utils.callbacks import CustomModelCheckpoint, CustomTensorBoard
from keras.optimizers import Adam,RMSprop
from utils.multi_gpu_model import multi_gpu_model
import tensorflow as tf
import keras
from keras.models import load_model
import matplotlib.pyplot as plt

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"



In [ ]:
def create_training_instances(
    train_annot_folder,
    train_image_folder,
    train_cache,
    valid_annot_folder,
    valid_image_folder,
    valid_cache,
    labels,
):
    # parse annotations of the training set
    train_ints, train_labels = read_annotations(train_annot_folder, train_image_folder, train_cache, labels)

    # parse annotations of the validation set, if any, otherwise split the training set
    if os.path.exists(valid_annot_folder):
        valid_ints, valid_labels = read_annotations(valid_annot_folder, valid_image_folder, valid_cache, labels)
    else:
        print("valid_annot_folder not exists. Spliting the trainining set.")

        train_valid_split = int(0.8*len(train_ints))
        np.random.seed(0)
        np.random.shuffle(train_ints)
        np.random.seed()

        valid_ints = train_ints[train_valid_split:]
        train_ints = train_ints[:train_valid_split]

    # compare the seen labels with the given labels in config.json
    max_box_per_image = max([len(inst['object']) for inst in (train_ints + valid_ints)])
    if len(labels) > 0:
        overlap_labels = set(labels).intersection(set(train_labels.keys()))

        print('Seen labels: \t'  + str(train_labels) )
        print('Given labels: \t' + str(labels))
        print('max_box_per_image: \t' + str(max_box_per_image))

        # return None, None, None if some given label is not in the dataset
        if len(overlap_labels) < len(labels):
            print('Label error in annotations')
            return None, None, None
    else:
        print('No labels are provided. Train on all seen labels.')
        print(train_labels)
        labels = train_labels.keys()

    max_box_per_image = max([len(inst['object']) for inst in (train_ints + valid_ints)])

    return train_ints, valid_ints, sorted(labels), max_box_per_image





In [ ]:
def create_callbacks(saved_weights_name, tensorboard_logs, model_to_save):
    makedirs(tensorboard_logs)
    
    early_stop = EarlyStopping(
        monitor     = 'loss', 
        min_delta   = 1e-5, 
        patience    = 5, 
        mode        = 'min', 
        verbose     = 1
    )
    checkpoint = CustomModelCheckpoint(
        model_to_save   = model_to_save,
        filepath        = saved_weights_name,# + '{epoch:02d}.h5', 
        monitor         = 'loss', 
        verbose         = 1, 
        save_best_only  = True, 
        mode            = 'min', 
        period          = 1
    )
    reduce_on_plateau = ReduceLROnPlateau(
        monitor  = 'loss',
        factor   = 0.1,
        patience = 3,
        verbose  = 1,
        mode     = 'min',
        epsilon  = 0.01,
        cooldown = 0,
        min_lr   = 0
    )
    tensorboard = CustomTensorBoard(
        log_dir                = tensorboard_logs,
        write_graph            = True,
        write_images           = True,
    )    
    return [early_stop, checkpoint, reduce_on_plateau]

In [ ]:
def create_model(
                nb_class, 
                anchors, 
                max_box_per_image, 
                max_grid, batch_size, 
                warmup_batches, 
                ignore_thresh, 
                multi_gpu, 
                saved_weights_name, 
                lr,
                grid_scales,
                obj_scale,
                noobj_scale,
                xywh_scale,
                class_scale  
            ):
    if multi_gpu > 1:
        with tf.device('/cpu:0'):
            template_model, infer_model = model(
                nb_class            = nb_class, 
                anchors             = anchors, 
                max_box_per_image   = max_box_per_image, 
                max_grid            = max_grid, 
                batch_size          = batch_size//multi_gpu, 
                warmup_batches      = warmup_batches,
                ignore_thresh       = ignore_thresh,
                grid_scales         = grid_scales,
                obj_scale           = obj_scale,
                noobj_scale         = noobj_scale,
                xywh_scale          = xywh_scale,
                class_scale         = class_scale
            )
    else:
        template_model, infer_model = model(
            nb_class            = nb_class, 
            anchors             = anchors, 
            max_box_per_image   = max_box_per_image, 
            max_grid            = max_grid, 
            batch_size          = batch_size, 
            warmup_batches      = warmup_batches,
            ignore_thresh       = ignore_thresh,
            grid_scales         = grid_scales,
            obj_scale           = obj_scale,
            noobj_scale         = noobj_scale,
            xywh_scale          = xywh_scale,
            class_scale         = class_scale
        )  

    # load the pretrained weight if exists, otherwise load the backend weight only
    if os.path.exists(saved_weights_name): 
        print("\nLoading pretrained weights.\n")
        template_model.load_weights(saved_weights_name)
    
    if multi_gpu > 1:
        train_model = multi_gpu_model(template_model, gpus=multi_gpu)
    else:
        train_model = template_model      

    #optimizer = Adam(lr=lr, clipnorm=0.001)
    optimizer = RMSprop(lr=lr, rho=0.9, epsilon=1e-08, decay=0.0,clipnorm=0.001)
    train_model.compile(loss=dummy_loss, optimizer=optimizer)             

    return train_model, infer_model


In [ ]:
train_annot_folder  = 'data/malaria_phone_dataset/train/'
train_image_folder  = 'data/malaria_phone_dataset/train/'
valid_annot_folder  = 'data/malaria_phone_dataset/valid/'
valid_image_folder  = 'data/malaria_phone_dataset/valid/'
train_cache_name    = 'data/train_malaria_phone_dataset.pkl'
valid_cache_name    = 'data/valid_malaria_phone_dataset.pkl'
labels              = ["mp"]
tensorboard_dir     = 'logs/'
num_anchors         = 9
anchors             = [19,22, 20,22, 22,23, 27,38, 28,28, 34,22, 39,31, 39,41, 49,48]

''''Achors for different image size range. 
    To get better results the range between min and max should be less than 128 
#[24,26, 33,47, 38,38, 40,29, 46,48, 50,60, 52,39, 59,52, 66,67]  net_size = 736
#[19,22, 20,22, 22,23, 27,38, 28,28, 34,22, 39,31, 39,41, 49,48]  net_size = 544
#[11,13, 14,15, 16,24, 18,18, 21,21, 22,14, 25,27, 27,21, 31,31]  net_size = 352
#[7,7, 10,8, 10,14, 10,11, 12,11, 13,15, 16,11, 17,16, 20,20]     net_size = 224 
'''


batch_size          = 16
min_input_size      = 480
max_input_size      = 608
net_size            = 544
grid_scales         = [4,2,1]
obj_scale           = 5
noobj_scale         = 1
xywh_scale          = 1
class_scale         = 1
downsample          = 32
saved_weights_name  = 'weights/weights_phone_dataset/weights_480-6080.h5'
warmup_epochs       = 3
train_times         = 10
valid_times         = 1
gpus                = ""
ignore_thresh       = 0.35
learning_rate       = 4.5e-4
epochs              = 10
debug               = 1



#Parse annotations 
train_ints, valid_ints, labels, max_box_per_image = create_training_instances(train_annot_folder,
                                                           train_image_folder,
                                                           train_cache_name,
                                                           valid_annot_folder,
                                                           valid_image_folder,
                                                           valid_cache_name,
                                                           labels)

#Generate batches for training andd validation    
train_generator = create_batch(
    instances           = train_ints, 
    anchors             = anchors,   
    labels              = labels,        
    downsample          = downsample , # ratio between network input's size and network output's size, 32 for YOLOv3
    max_box_per_image   = max_box_per_image,
    batch_size          = batch_size, 
    min_net_size        = min_input_size, 
    max_net_size        = max_input_size ,
    net_size            = net_size,
    shuffle             = True, 
    jitter              = True, 
    norm                = None
)
    
valid_generator = create_batch(
    instances           = valid_ints, 
    anchors             = anchors,   
    labels              = labels,        
    downsample          = downsample , # ratio between network input's size and network output's size, 32 for YOLOv3
    max_box_per_image   = max_box_per_image,
    batch_size          = batch_size ,
    min_net_size        = min_input_size, 
    max_net_size        = max_input_size,
    net_size            = net_size,
    shuffle             = True, 
    jitter              = False, 
    norm                = normalize
)



In [ ]:
#Sanity check image and annotations. Ensure that image augumentaions corrects the bouding boxes
image = train_generator [0][0][0][0]
plt.figure(figsize=(8,8))
plt.imshow(image.astype('uint8'))
#plt.axis('off')
plt.show()

In [ ]:
#Create model
if os.path.exists(saved_weights_name): 
    warmup_epochs = 0
warmup_batches = warmup_epochs * train_times*len(train_generator)  

os.environ['CUDA_VISIBLE_DEVICES'] = gpus
multi_gpu = len(gpus.split(','))

train_model, infer_model = create_model(
    nb_class            = len(labels), 
    anchors             = anchors, 
    max_grid            = [max_input_size,max_input_size], 
    max_box_per_image   = max_box_per_image,    
    batch_size          = batch_size, 
    warmup_batches      = warmup_batches,
    ignore_thresh       = ignore_thresh,
    multi_gpu           = multi_gpu,
    saved_weights_name  = saved_weights_name,
    lr                  = learning_rate,
    grid_scales         = grid_scales,
    obj_scale           = obj_scale,
    noobj_scale         = noobj_scale,
    xywh_scale          = xywh_scale,
    class_scale         = class_scale
)

#start trainng

callbacks = create_callbacks(saved_weights_name, tensorboard_dir, infer_model)
i =0
while i<=15:
    
    #reload the bathes fr each training iteration
    train_generator = create_batch(
    instances           = train_ints, 
    anchors             = anchors,   
    labels              = labels,        
    downsample          = downsample , 
    max_box_per_image   = max_box_per_image,
    batch_size          = batch_size, 
    min_net_size        = min_input_size, 
    max_net_size        = max_input_size ,
    net_size            = net_size,
    shuffle             = True, 
    jitter              = True, 
    norm                = normalize)


    saved_weights  = saved_weights_name[:-3]+'_'+str(i)+'.h5'
    
    if i>0:
        epochs = 50
        warmup_epochs = 0
        checkPoint= saved_weights_name[:-3]+'_'+str(i-1)+'.h5'
        train_model.load_weights(checkPoint)
        print('Saved model loaded: ',  checkPoint)

    train_model.fit_generator(
        generator        = train_generator, 
        steps_per_epoch  = len(train_generator) * train_times, 
        epochs           = epochs+ warmup_epochs, 
        verbose          = 1,
        validation_data  = valid_generator,
        validation_steps = len(valid_generator) * valid_times,
        callbacks        = callbacks, 
        workers          = 10,
        max_queue_size   = 20,
        shuffle         =  True
    )
    i +=1
    
    
    train_model.save_weights(saved_weights)
    
    

    #eavlate MAP for entire validations set
    average_precisions = evaluate(infer_model, 
                                  valid_generator,
                                  iou_threshold=0.3,
                                  obj_thresh=0.5,
                                  nms_thresh=0.05,
                                  net_h=net_size,
                                  net_w=net_size)

    # print the score
    for label, average_precision in average_precisions.items():
        print(labels[label] + ': {:.4f}'.format(average_precision))
    print('mAP: {:.4f}'.format(sum(average_precisions.values()) / len(average_precisions))) 